In [176]:
from serde import serde
from serde.json import to_json, from_json
from dataclasses import dataclass
from time import sleep
from scrapping import Voiture
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import OneHotEncoder

In [177]:
with open("donnees.json", "r") as fichier:
    contenu_fichier = fichier.read()

In [178]:
reconstitution = from_json(list[Voiture],contenu_fichier)

In [179]:
df = pd.DataFrame(reconstitution)
df = df.replace('NA', np.NaN)

In [180]:
df_mod = pd.get_dummies(df["modele"])
df_garkil = pd.get_dummies(df["garantie_kilometrage"])
df_carbu = pd.get_dummies(df["carburant"])
df_bdv = pd.get_dummies(df["boite_de_vitesse"])
df_couleur = pd.get_dummies(df["couleur"])
df_sil = pd.get_dummies(df["silhouette"])
df_trans = pd.get_dummies(df["transmission"])
df_gar = pd.get_dummies(df["garantie"])
df_utilprec = pd.get_dummies(df["utilisation_prec"])

In [181]:
df = pd.concat([df, df_mod, df_garkil, df_carbu, df_bdv, df_couleur, df_sil,df_trans, df_gar, df_utilprec], axis = 1)

In [182]:
del df['modele']
del df['garantie_kilometrage']
del df['carburant']
del df['boite_de_vitesse']
del df['couleur']
del df['silhouette']
del df['transmission']
del df['garantie']
del df['utilisation_prec']

In [183]:
df_EN = df.dropna()

In [184]:
y = df_EN['prix'].to_numpy()
X = df_EN.loc[:, df_EN.columns != 'prix'].to_numpy() # tout sauf le prix

In [185]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y)

In [186]:
en = ElasticNet()
en_gs = GridSearchCV(
    en,
    {
        "alpha": [2 ** p  for p in range(-6, 6)],
        "l1_ratio": (0.01, 0.25, 0.5, 0.75, 1),
    }
)
en_gs.fit(X_tr, y_tr) #problème vient du fit
en_df = pd.DataFrame(en_gs.cv_results_)        

c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.798e+08, tolerance: 2.520e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.950e+08, tolerance: 2.543e+06
  model = cd_fast.enet_coordinate_descent(
c:\Users\mathi\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:634: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the sca

In [187]:
en_gs.best_params_, en_gs.best_score_

({'alpha': 0.0625, 'l1_ratio': 0.5}, 0.9213771874895468)